## Lab 7 - Build a Dashboard Application with Plotly Dash

## Instructions
https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/lab_theia_plotly_dash.md.html

https://www.coursera.org/learn/applied-data-science-capstone/ungradedLti/EtmJx/hands-on-lab-build-an-interactive-dashboard-with-ploty-dash
___

After visual analysis using the dashboard, you should be able to obtain some insights to answer the following five questions:

* Which site has the largest successful launches?  
* Which site has the highest launch success rate?  
* Which payload range(s) has the highest launch success rate?  
* Which payload range(s) has the lowest launch success rate?  
* Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?

This is the dataset we're working with:

In [2]:
import os
import pandas as pd

os.listdir(os.getcwd())

csv_path = 'spacex_launch_dash.csv'
df = pd.read_csv(csv_path)

df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


___
## Dashboard Application Code:

*To run the code, just run the cell below, and click on the link it produces underneath. It may take a second for it to execute and show the link.*

In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

min_value = 0
max_value = 100

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                    options=[
                                        {'label': 'All Sites', 'value': 'ALL'},
                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                    ],
                                    value = 'ALL',
                                    placeholder = "Select a Launch Site here",
                                    searchable = True
                                    ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range                                
                                dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000, marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'}, value = [min_value, max_value]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site): 
    filtered_df = spacex_df
    if entered_site == 'ALL':
        #here I need to make a dataframe containing the total successful launches for each location to use in piechart below
        fig = px.pie(filtered_df, values='class', names='Launch Site', title='Total Successful Launches by Site')
        return fig
    else:
        #Filter the dataframe to get data for the selected launch site.
        #Then, build a pie chart based on that. We return the pie chart.
        summing_df = filtered_df[filtered_df['Launch Site'] == entered_site] #contains all of the CCAFS LC 40 rows

        summing_df = summing_df[['class']].value_counts()
        zerosum = summing_df[0]
        onesum = summing_df[1]

        my_dict = {'Class': [0,1], 'Sum': [zerosum, onesum]}
        summing_df = pd.DataFrame.from_dict(my_dict)
        title_string = "Total Success Launches for " + str(entered_site) + " Launch Site"
        fig = px.pie(summing_df, values='Sum', names='Class', title= title_string)

        # return the outcomes piechart for a selected site
        return fig

        
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
             [Input(component_id='site-dropdown', component_property='value'), 
              Input(component_id="payload-slider", component_property="value")])

def get_scatter_plot(entered_site, slider_range):
    
    #print("Entered site is: \n", entered_site) #IS A STRING (The selected site; ex: 'ALL' or 'CCAFLCS')
    #print("Slider range is: \n", slider_range) #is a LIST (The range slider)
    
    filtered_df = spacex_df

    if (entered_site == 'ALL'):
        scatter_plot = px.scatter(filtered_df, x = 'Payload Mass (kg)', y = 'class', color="Booster Version Category", range_x = slider_range, title = "Correlation between Payload and Success for all Launch Sites")

    else:
        graph_title = "Correlation between Payload and Success for " + str(entered_site) + " Launch Site"
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site] #filtered_df contains all data rows of selected site
        scatter_plot = px.scatter(filtered_df, x = 'Payload Mass (kg)', y = 'class', color="Booster Version Category", range_x = slider_range, title = graph_title)
        
    return scatter_plot




# Run the app
if __name__ == '__main__':
    app.run_server()


Dash is running on http://127.0.0.1:8050/



<ipython-input-1-501ec7f0b75d>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-1-501ec7f0b75d>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Aug/2022 12:28:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Aug/2022 12:28:12] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_6_1m1659644661.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Aug/2022 12:28:12] "GET /_dash-component-suites/dash/deps/react@16.v2_6_1m1659644661.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Aug/2022 12:28:12] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_6_1m1659644661.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Aug/2022 12:28:12] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_6_1m1659644661.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Aug/2022 12:28:12] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_6_1m1659644661.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Aug/2022 12:28:12] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_6_1m1659644661.min.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Aug/2022 12:28:12] "GET /_dash-component-s

___
## Finding Insights Visually
Now with the dashboard completed, you should be able to use it to analyze SpaceX launch data, and answer the following questions:

**Which site has the largest successful launches?**  
* Class = 1 is a successful landing. Class = 0 is an unsuccessful landing. KSC LC 39A has the highest amount of successful landings. It comprises 41.7% of all successful landings out of all of the launch sites. 

**Which site has the highest launch success rate?**  
* Launches from KSC LC 39A have a 76.9% success rate.  

**Which payload range(s) has the highest launch success rate?**  
* Payload ranges from 2500kg to 5000kg have the highest launch success rate.

**Which payload range(s) has the lowest launch success rate?**  
* Payload range 0kg to 2500kg has the lowest launch success rate

**Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?**  
* FT has the highest launch success rate.